# DIA Tutorial: Exposure Information
This notebook is compatible with `desc-stack` kernel.

Obtain MJD, filter, magToInstFlux translation, background level, gain, zeropoint information of a calexp.

In [1]:
import lsst
from scipy.stats import sigmaclip
import numpy as np
from lsst.daf.persistence import Butler

In [ ]:
calexp_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-calexp-v1'
calexp_butler = Butler(calexp_repo)
# get exp and photocalib
calexp_id = {'visit': 1013665, 'detector': 79, 'filter': 'i'}
calexp_exp = calexp_butler.get('calexp', calexp_id)
calexp_photocalib = calexp_butler.get('calexp_photoCalib', calexp_id)
# get the source table measured from the calexp exposure
calexp_src = calexp_butler.get('src', calexp_id)
# mjd
# https://community.lsst.org/t/how-should-one-efficiently-get-the-mjd-for-a-source-catalog/3733/2
info = calexp_exp.getInfo()
# ap_corr = info.getApCorrMap()
mjd = info.getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD)
# lsst.daf.base.DateTime.UTC
print(f'MJD: {mjd}')
# filter
print('filter')
filt = calexp_exp.getFilter()
print(f'filter: Id {filt.getId()}, Name {filt.getName()}, Names {filt.getNames()}' )

# detector
print('detector')
det = calexp_exp.getDetector()
print(f'detector: Name {det.getName()}, Id {det.getId()}, Type {det.getType()}, Serial {det.getSerial()}')

# sky background estimated using image plane
img = calexp_exp.getImage().array
img = img[~np.isnan(img)]
clipped, _, _ = sigmaclip(img, low=5, high=5)
var_of_img = np.var(clipped)
print(f'variance of image: {var_of_img}')

# mean variance plane
var = calexp_exp.getVariance().array
var = var[~np.isnan(var)]
clipped, _, _ = sigmaclip(var, low=5, high=5)
mean_of_var = np.mean(clipped)
print(f'mean of variance plane: {mean_of_var}')

inst_to_nJ = photo_calib.instFluxToNanojansky(1)
print(f'instFlux to nanoJansky: {inst_to_nJ}')
print('Is this equivalent to gain?')

# mag zp
photo_calib = calexp_exp.getPhotoCalib()
mag_zp = photo_calib.getInstFluxAtZeroMagnitude()
print(f'mag zp: {mag_zp}')